**formatPropertyMissing.ipynb**

Deprecated concept with disallowed annotations or axioms.

**Documentation:** https://github.com/edamontology/edamverify/blob/master/docs/formatPropertyMissing.md

Set constants for script return values. Load EDAM_dev.owl from GitHub into an RDF graph.

In [4]:
import sys
from rdflib import ConjunctiveGraph, Namespace

# Constants for script return value as per https://github.com/edamontology/edamverify.
NOERR = 0
INFO  = 1
WARN  = 2
ERROR = 3

#Load EDAM_dev.owl from GitHub into an RDF graph.
print("Loading graph ...", end="")
g = ConjunctiveGraph()
g.load('https://raw.githubusercontent.com/edamontology/edamontology/master/EDAM_dev.owl', format='xml')
# g.load('EDAM_dev.owl', format='xml')
g.bind('edam', Namespace('http://edamontology.org#'))
print("done!")



Loading graph ...done!


Define SPARQL query to retrieve ID, term, and all properties that are mandatory or recommended for Format concepts. Run the query.

**NB.1:** BASE is used to define the define the default namespace (for various elements below).

**NB.2:** A concept is considered to be a format if it has a property ``<oboInOwl:inSubset>`` whose value contains the string ``formats``, e.g. ``<oboInOwl:inSubset rdf:resource="http://purl.obolibrary.org/obo/edam#formats"/>``.

**NB.3:**
Subsets in EDAM are (currently) defined in one of two ways:

``<oboInOwl:inSubset rdf:resource="http://purl.obolibrary.org/obo/edam#data" />``

``<oboInOwl:inSubset>concept_properties />``

this complicates the logic slightly.

In [5]:
# Compile SPARQL query
query_term = """
BASE <http://edamontology.org/>
SELECT ?id ?term ?subsetpath ?is_format_of ?documentation ?file_extension ?media_type ?example ?information_standard ?ontology_used ?organisation WHERE
{
?id rdfs:label ?term .
?id oboInOwl:inSubset ?subsetpath .
OPTIONAL {?id :is_format_of ?is_format_of .}
OPTIONAL {?id :documentation ?documentation .}
OPTIONAL {?id :file_extension ?file_extension .}
OPTIONAL {?id :media_type ?media_type .}
OPTIONAL {?id :example ?example .}
OPTIONAL {?id :information_standard ?information_standard .}
OPTIONAL {?id :ontology_used ?ontology_used .}
OPTIONAL {?id :organisation ?organisation .}
}
"""

# Run SPARQL query and collate results
err = NOERR
errfound = False
report = list()
results = g.query(query_term)

Analyse results of query.

In [6]:
# Error message building function
def report_append(err_new, msg):
    """
    Writes a report of an error and sets the error level appropriately.
    err: the error level to set
    msg: the message to write
    """
    global err
    global errfound
    global report
    if err_new > err:
        err = err_new
    errfound = True
    report.append(msg)

for r in results :
#    print(str(r['id']), str(r['term']), str(r['ext']), str(r['exact_syn']))
    id   = str(r['id'])
    term = str(r['term'])
    subsetpath  = str(r['subsetpath'])

    is_format_of = str(r['is_format_of'])
    documentation = str(r['documentation'])
    file_extension = str(r['file_extension'])
    media_type = str(r['media_type'])
    example = str(r['example'])
    information_standard = str(r['information_standard'])
    ontology_used = str(r['ontology_used'])
    organisation = str(r['organisation'])


    # Extract subset without path, e.g. "data"
    pos = subsetpath.rfind("#")
    if pos == -1:
        # subset path does not contain '#'
        subset = subsetpath
    else:
        subset = subsetpath[pos+1:]

    # Check for (and skip) obsolete concepts
    if subset == "formats":
        if is_format_of == "None":
            report_append(ERROR, "ERROR Missing property (is_format_of) on ::: " + id +  ' (' + term + ')')
            continue
        if documentation == "None":
            report_append(ERROR, "ERROR Missing property (documentation) on ::: " + id +  ' (' + term + ')')
            continue
        if file_extension == "None":
            report_append(WARN, "WARN Missing property (file_extension) on ::: " + id +  ' (' + term + ')')
            continue
        if media_type == "None":
            report_append(INFO, "INFO Missing property (media_type) on ::: " + id +  ' (' + term + ')')
            continue
        if example == "None":
            report_append(INFO, "INFO Missing property (example) on ::: " + id +  ' (' + term + ')')
            continue
        if information_standard == "None":
            report_append(INFO, "INFO Missing property (information_standard) on ::: " + id +  ' (' + term + ')')
            continue
        if ontology_used == "None":
            report_append(INFO, "INFO Missing property (ontology_used) on ::: " + id +  ' (' + term + ')')
            continue
        if organisation == "None":
            report_append(INFO, "INFO Missing property (organisation) on ::: " + id +  ' (' + term + ')')
            continue

Write report and return approriate value.

In [7]:
# Return exit code (raises exception)
if errfound:
    print('Invalid property found for these obsolete concepts:')
    print("\n".join(report))
    sys.exit(WARN)
else:
    print("No issues found.")
    sys.exit(NOERR)

Invalid property found for these obsolete concepts:
ERROR Missing property (is_format_of) on ::: http://edamontology.org/format_3007 (PSL)
ERROR Missing property (is_format_of) on ::: http://edamontology.org/format_1430 (Phylip continuous quantitative characters)
ERROR Missing property (is_format_of) on ::: http://edamontology.org/format_2065 (Protein structure report (quality evaluation) format)
ERROR Missing property (is_format_of) on ::: http://edamontology.org/format_3245 (Mass spectrometry data format)
ERROR Missing property (is_format_of) on ::: http://edamontology.org/format_1935 (GCG)
ERROR Missing property (is_format_of) on ::: http://edamontology.org/format_2567 (completely unambiguous pure)
ERROR Missing property (is_format_of) on ::: http://edamontology.org/format_1933 (FASTQ-solexa)
ERROR Missing property (is_format_of) on ::: http://edamontology.org/format_1954 (Pearson format)
ERROR Missing property (is_format_of) on ::: http://edamontology.org/format_1216 (unambiguous p

SystemExit: 2

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
